# Assigment 1 : Obstacle avoidance and  Motion Planning - Hugo Millet

The objective of this notebook is to develop parts of the code that needs further explanations

## Part 1 : Creating the occupancy map

In my code, the occupancy grid is stored in the variable initgrid. Its parameters are resolution, and size, which are explained in the commentaries of the code.
Each cell can have 3 values, 0 if it's free, 1 if it's an obstacle, and -1 if there is the robot in that cell

The occupancy grid is therefore modified in the two callbacks from the suscribers to Odometry and Scan. The odometry one fills the position of the robot and the scan one the position of the obstacles.

#### 1.1. Filling the position of the robot in the occupancy grid

The part of the code responsible for this operation is shown below :

In [ ]:
xpos = odom.pose.pose.position.x  # get the X measurement in m of the robot pos
ypos = odom.pose.pose.position.y  # get the Y measurement in m of the robot pos
yaw = quater_to_euler(odom.pose.pose.orientation)[2] # get the yaw in radian of the robot

# get the X coordinate of the cell where the robot is
cellX = int((resolution-1)/2) + int(xpos/size)
# get the Y coordinate of the cell where the robot is
cellY = int((resolution-1)/2) + int(ypos/size)

reset_robot_pos(initgrid) # removing the previous robot pos
initgrid[cellX, cellY] = -1 # setting the robot pos in the cell

Basically, what it is doing is get the positions in meters of the robot from the Odometry msg, and converting it in terms of cell number, which depends on the size of the cell.
For practical reasons, I also chose to set the [0,0] odometry point in the center of my grid, which is why I also had the term int((resolution-1)/2). I usually chose resolutions values to be odd, to have a cell perfectly in the center and this explains the presence of the -1. 

#### 1.2. Filling the position of the obstacles in the occupancy grid

The part of the code responsible for this operation is shown below :

In [ ]:
    ranges = scan.ranges
    for i in range(len(ranges)):
        if ranges[i] < scan.range_max:
            angle_wf = yaw + i*scan.angle_increment # angle of the range in the world frame
            while(angle_wf > math.pi*2): # keeping values in the [0, 2pi]
                angle_wf -= math.pi*2
            dx_bot_to_range = ranges[i]*math.cos(angle_wf) # distance in x from the robot to the obstacle in m 
            dy_bot_to_range = ranges[i]*math.sin(angle_wf) # distance in y from the robot to the obstacle in m

            cell_X_obstacle = cellX+int(dx_bot_to_range/size) # X coordinate of the obstacle
            cell_Y_obstacle = cellY+int(dy_bot_to_range/size) # Y coordinate of the obstacle
            if cell_X_obstacle < resolution and cell_Y_obstacle < resolution: # if the obstacle is not out of the grid
                initgrid[cell_X_obstacle, cell_Y_obstacle] = 1 # setting the value for the obstacle cell


First, it's getting the ranges parameter from the LaserScan msg. For each of these values, it is transposing the angle from robot frame to world frame using the yaw value of the robot obtained from Odometry. Them, it is transposing the range value along the X and Y axis of the world frame.
Them, as for the robot position, it transpose meters value into numbered cells value and set the value to 1.

## Part 2 : Creating the field map

The field map has the same X and Y dimensions as the occupancy one, but each of its cells will contains two values : one for the field in X and one for the field in Y.
My first try to code this was to fill the entire map with values but a big drawback from this method was the calculating power it was demanding and therefore was making the program very slow to execute.
The new and final approach I went with was to fill only the grid in which the robot is currently, which is much quicker.
For this, I created two functions called create_att_field() and create_rep_field(mygrid), this last one taking the initgrid as a parameter cause we will need the positions of the obstacles.

#### 2.1. Creating the attractive field

The function is shown below :

In [ ]:
def create_att_field():
    global cellgX
    global cellgY
    global att_field_grid

    goalx = rospy.get_param("/goal_x")
    goaly = rospy.get_param("/goal_y")

    # Converting goal parameters from meters to cells number
    cellgX = int((resolution-1)/2) + int(goalx/size)
    cellgY = int((resolution-1)/2) + int(goaly/size)
    
    att_field_grid[cellX, cellY, 0] = k_att*(cellgX-cellX) # Attractive field on the x axis is created
    att_field_grid[cellX, cellY, 1] = k_att*(cellgY-cellY) # Attractive field on the y axis is created

    return att_field_grid

We first obtain the goal parameters in meters from the launch file, converting them in number cells, and filling the cell in which the robot is with the mathematical formula of the attractive field, before returning the grid.

#### 2.2. Creating the repulsive field

The function is shown below :

In [ ]:
def create_rep_field(mygrid):
    global cellX
    global cellY
    global rep_field_grid

    fieldX = 0
    fieldY = 0

    for a in range(-Qlim, Qlim+1):
        for b in range(-Qlim, Qlim+1):
            # checking in the Qlim*Qlim square around the cell
            dcell = math.sqrt(a**2+b**2) # calculating the distance in terms of cells from the supposed obstacle cell
            if cellX+a >= 0 and cellY+b >= 0 and cellX+a < resolution and cellY+b < resolution:
            #if the checked cell is still in the grid
                if dcell < Qlim and dcell != 0 and mygrid[cellX+a, cellY+b] == 1:
                # eliminating non concerned-cells of the square, the field cell itself and only if an obstacle cell is detected
                    fieldX += k_rep*(((1/dcell)-(1/Qlim))/(dcell**2))*(-a/dcell) # adding the repulsive field emitted from the obstacle cell
                    fieldY += k_rep*(((1/dcell)-(1/Qlim))/(dcell**2))*(-b/dcell) # adding the repulsive field emitted from the obstacle cell

    rep_field_grid[cellX, cellY, 0] = fieldX # 0 if no obstacle, total field if there is
    rep_field_grid[cellX, cellY, 1] = fieldY # 0 if no obstacle, total field if there is

    return rep_field_grid

This function functions in this way :
Around the robot cell, we check all the cells in the square around it, the side of the square being 2xQlim, our limit parameter for the repulsive field.
Then, for each cell of this square, we check if that cell is still within the grid limit, and with the second if we check several things :
- First, not all the cells in the square are concerned by the repulsive field, some are still too far. We eliminate that one.
- Then, we don't check the robot cell itself
- Finally, we check if there is an obstacle in one of the cell
If so, we add the field created by the obstacle cell to the robot cell. If there is multiple obstacle cells in reach, each fields add before being applied.

Finally, we create the total field with the following line at the end of the scan callback :

In [ ]:
fieldgrid = create_att_field() + create_rep_field(initgrid) # creating field

## Part 3 : Commanding the robot

Since commanding the robot is directly dependant of its position, it makes more sense to place this in the odometry callback. 

There are several steps.

In [ ]:
obj_angle_wf = math.atan2(fieldgrid[cellX,cellY,1], fieldgrid[cellX,cellY,0]) # objective angle in the worldframe
    if obj_angle_wf < 0:
        obj_angle_wf += 2*math.pi
    if yaw < 0:
        yaw += 2*math.pi  

First, we convert the X and Y field values of the cell in term of angle, with the atan2 function. It corresponds to the angle in the world frame in which the robot should move. Originally, the conversion gives us an angle in the [-pi;pi] interval, but for practical reasons I transposed it in the [0;2pi] interval, and did the same to the yaw angle of the robot.

In [ ]:
if yaw < obj_angle_wf - 0.1 or yaw > obj_angle_wf + 0.1: # if the yaw is too different than the objective angle
        cmd.linear.x = 0 # no linear speed
        cmd.angular.z = math.copysign(0.8, obj_angle_wf-yaw) # rotating in the orientation of the difference

Then, if our robot yaw orientation is not the same as our objective angle with a slight tolerance (here + or - 0.1), we move the robot along the z axis by applying a command on the cmd.angular.z of the Twist message that we publish. The speed is constant but we go to the shorter direction by getting the sign of the difference between our objective angle and our yaw.


In [ ]:
else: #if the robot is in the right orientation
        cmd.angular.z = 0 # no more angular speed
        normF = math.sqrt(fieldgrid[cellX,cellY,1]**2+fieldgrid[cellX,cellY,0]**2) # norm of the field force
        if normF < 10 and normF > 0: # if we're close enough from the goal and not at the goal
            cmd.linear.x = 0.1 # constant speed
        else: # if not
            cmd.linear.x = 0.01*normF # linear speed proportional to the norm of the field force

Once our robot is in the correct orientation, we can make it move forward.
If we are far enough from the goal or at the goal (the else condition), the speed is proportional to the intensity of the field force.
If we are close enough of the goal, doing it this way would not be practical cause the speed would decrease a lot and the robot would take a lot of time to reach the goal. Therefore, once it is close enough, we set it to a constant speed.

## Part 4 : Results and analysis

By launching the progam with the goals parameters set, the robot eventually reaches the goal, while avoiding the cylinder shaped-obstacle.
I tried with different goal points and some of them worked, others didn't.

A remark I noticed during my attempts is that the repulsive field formula may not be optimal : one cell still in the Qlim limit but far from the obstacle will have almost no impact on the total field but the next cell next to it, closer to the obstacle will have a very strong impact. Practically, this cause the robot to go successively closer and further from the obstacle in an almost perpendicular way. I think this is due to the term of the distance to the obstacle, dcells, that is in the cube power at the denominator of the formula, which causes abrupt changes from one cell to another.

The attempts that didn't work were either due to this imperfection, that makes the robot blocks with the obstacle, or a similar case but due to the principle of the field method itself : when an obstacle is perfectly perpendicular to the direction of the robot should go to go to the goal, the field of the obstacle would be in the opposite direction of the attracting field, therefore condeming the robot to go back and forth in front of the obstacle endlessly.
Another case of failed attempt was because of parasites obstacles on my occupancy map ; sometimes the map is filled with an obstacle cell where there isn't, and since my resolution is pretty low it has a non-neglecting impact if the robot cross the way of this parasite obstacle cell.

There are some ameliorations that could improve the program :
- Having a better resolution for the map, but we are limited by the virtual machine principle that TheConstruct uses but also the way my code is built
- A more precise tuning of the attractive and repulsive fields coefficients
- Adjusting the angular speed depending on how much angle we have to move
